In [1]:
from pyther_classes import *
from pyther_fn import *

In [2]:
# set path
dirPath = '/mnt/c/Users/lvlah/linux/ac_lab/pyther/'

In [3]:
# load interactome
intObj = InteractomefromTSV(dirPath + 'data/stoch-10000_pruned.tsv', 's10k-net')

In [4]:
# load ges
gesObj = anndata.read_csv(dirPath + 'data/stoch-10000_ccle-ges.tsv', delimiter = '\t')

In [5]:
# run aREA
nesMat = aREA(gesObj[:, 0:3], intObj)

ValueError: Dot product shape mismatch, (1927, 3783) vs (3685, 3)

In [ ]:
## load data
intObj = InteractomefromTSV('/mnt/c/Users/lvlah/linux/ac_lab/data/pyther_data/test-net.tsv', 'testNet')
gesDF = anndata.read_csv('/mnt/c/Users/lvlah/linux/ac_lab/data/pyther_data/gtex-aorta_ges.tsv', delimiter = '\t')
r = 'ENSG00000000003'

In [ ]:
aREA(gesDF[:, 0:3], intObj)